## QHyper usecase

### Create instance of problem

In [6]:
from QHyper.problems.knapsack import KnapsackProblem
from QHyper.problems.tsp import TSPProblem

PROBLEM_TYPE = 'knapsack' # 'tsp'


# Each problem needs different parameters, because it depends on the number
# of variables and constraints
# Number of hyper_optimizer_bounds depends on the number of constraints,
# because each constraint requires one weights, and objective function also
# requires one
 
if PROBLEM_TYPE == 'knapsack':
    # Create knapsack with 3 items: 2 with weights 1 and value 2 - (1, 2),
    # and one with weight 1 and value 1 - (1, 1)
    problem = KnapsackProblem(max_weight=2, items=[(1, 2), (1, 2),(1, 1)])
    print(problem.knapsack.items)
    
    params_cofing = {
        'angles': [[0.5]*5, [1]*5],
        'hyper_args': [1, 2.5, 2.5],
    }
    hyper_optimizer_bounds = 3*[(1, 10)]

elif PROBLEM_TYPE == 'tsp':
    # Create Traveling Salesmam Problem with 3 cities
    problem = TSPProblem(
        number_of_cities=3,
    )
    params_cofing = {
        'angles': [[0.5]*5, [1]*5],
        'hyper_args': [1, 2, 2, 2, 2],
    }
    hyper_optimizer_bounds = 5*[(1, 10)]

[Item(weight=1, value=2), Item(weight=1, value=2), Item(weight=1, value=1)]


In [7]:
print(f"Variables used to describe objective function"
      f"and constraints: {problem.variables}")
print(f"Objective function: {problem.objective_function}")
print("Constraints:")
for constraint in problem.constraints:
    print(f"    {constraint}")

Variables used to describe objective functionand constraints: (x0, x1, x2, x3, x4)
Objective function: {('x0',): -2, ('x1',): -2, ('x2',): -1}
Constraints:
    {('x3',): -1, ('x4',): -1, (): 1}
    {('x0',): -1, ('x1',): -1, ('x2',): -1, ('x3',): 1, ('x4',): 2}


### Use VQA to solve knapsack problem

In [8]:
from QHyper.solvers.vqa.base import VQA
from QHyper.solvers import Solver

In [9]:
# Simple quantum circuit without optimzers will be used to test the results
# WF-QAOA is choosen becasue this PQC has most suitable evaluation function

tester_config = {
    'pqc': {
        'type': 'wfqaoa',
        'layers': 5,
    }
}

tester = VQA(problem, config=tester_config)

In [10]:
# Create a VQA instance with HQAOA as PQC and scipy optimizer
# This can be done in two various way
# 1. Providing dict with config (usefull to save experiment confing in e.g JSON)

solver_config = {
    "solver": {
        "type": "vqa",
        "args": {
            "config": {
                "optimizer": {
                    "type": "scipy",
                    "maxfun": 200,
                },
                "pqc": {
                    "type": "wfqaoa",
                    "layers": 5,
                }
            }
        }
    }
}

vqa = Solver.from_config(problem, solver_config)

# 2. Providing actual isntance of each class like VQA and Optimizer
from QHyper.solvers.vqa.pqc.h_qaoa import HQAOA
from QHyper.optimizers import ScipyOptimizer

vqa = VQA(problem, HQAOA(layers=5), ScipyOptimizer(maxfun=200))

In [11]:
# Run VQA with provided initial parameters
best_params = vqa.solve(params_cofing)
print(f"Best params: {best_params}")

Best params: {'angles': array([3.73821011, 0.27008228, 1.62472153, 2.00362456, 4.41866799,
       1.51786833, 0.40470488, 0.69482525, 0.        , 2.47660795]), 'hyper_args': array([1.41431361, 3.09335526, 4.43744609])}


In [16]:
# Here created ealier tester will be used to evaluate results

best_results = tester.evaluate(best_params)
print(f"Best results: {best_results}")
print(f"Params used for optimizer:\n{best_params['angles']},\n"
      f"and params used for hyper optimizer: {best_params['hyper_args']}")

Best results: -0.827832587433078
Params used for optimizer:
[3.73821011 0.27008228 1.62472153 2.00362456 4.41866799 1.51786833
 0.40470488 0.69482525 0.         2.47660795],
and params used for hyper optimizer: [1.41431361 3.09335526 4.43744609]


#### Using hyper optimizers

In [14]:
# Additionally other optimizer can be used to tune some parameters, in below
# example, Random optimzier will change weights (hyper_args) and choose ones
# that gives the best results after runnign 200 iteration of scipy minimizer

solver_config = {
    "solver": {
        "type": "vqa",
        "args": {
            "config": {
                "optimizer": {
                    "type": "scipy",
                    "maxfun": 200,
                },
                "pqc": {
                    "type": "wfqaoa",
                    "layers": 5,
                }
            }
        }
    },
    "hyper_optimizer": {
        "type": "random",    
        "processes": 5,
        "number_of_samples": 100,
        "bounds": hyper_optimizer_bounds
    }
}
vqa = Solver.from_config(problem, solver_config)

best_params_h = vqa.solve(params_cofing)

100%|██████████| 100/100 [10:06<00:00,  6.07s/it]


In [17]:
# Again, final results will be evaluated with tester

best_results = tester.evaluate(best_params_h)
print(f"Best results: {best_results}")
print(f"Params used for optimizer:\n{best_params_h['angles']},\n"
      f"and params used for hyper optimizer: {best_params['hyper_args']}")

Best results: -0.8092853930381896
Params used for optimizer:
[[1.642333   0.08750181 0.         0.12856457 2.49365188]
 [0.93903123 0.62302204 1.10216079 0.41515657 1.45091435]],
and params used for hyper optimizer: [1.41431361 3.09335526 4.43744609]
